In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

In [2]:
service = Service()  
options = webdriver.ChromeOptions()
options.add_argument("--disable-application-cache")  # disable cache
options.add_argument("--disk-cache-size=0")          # disable disk cache
options.add_argument("--start-maximized")

In [3]:
# Launch Driver
driver = webdriver.Chrome(service=service, options=options)

In [4]:
driver.get("https://creattie.com/lottie-animated-icons/all?type=free&orderBy=order&page=1")
time.sleep(2)

In [5]:
def js_click(el):
    # Scroll to center and click with a real MouseEvent that bubbles across shadows
    driver.execute_script("""
      const el = arguments[0];
      el.scrollIntoView({block: 'center', inline: 'nearest'});
      const ev = new MouseEvent('click', {bubbles: true, cancelable: true, composed: true});
      el.dispatchEvent(ev);
    """, el)

## Sign IN

In [6]:
sign_in_btn = driver.find_element(By.ID, "topSignIn")
sign_in_btn.click()

In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# wait until email and password fields are visible
wait = WebDriverWait(driver, 100)

email_input = wait.until(EC.element_to_be_clickable((By.ID, "email")))
password_input = wait.until(EC.element_to_be_clickable((By.ID, "password")))

# fill in credentials
email_input.clear()
email_input.send_keys("spmalla8@gmail.com")

password_input.clear()
password_input.send_keys("spmalla8848")

# click the sign-in button
login_btn = wait.until(EC.element_to_be_clickable((By.ID, "login-submit")))
login_btn.click()

## Item click

In [8]:
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time

wait = WebDriverWait(driver, 100)

while True:
    # --- 1) Process all blocks on current page ---
    container = wait.until(
        EC.presence_of_element_located((By.CLASS_NAME, "cr-product-container-icon"))
    )
    blocks = container.find_elements(By.CLASS_NAME, "cr-lottie-block")
    block_length = len(blocks)
    print(f"\nTotal blocks found on this page: {block_length}")

    for index in range(block_length):
        if (index + 1) % 40 == 0 and index != 0:
            print("Taking a longer break to avoid rate limits...")
            # scroll
            driver.execute_script("window.scrollTo(0, 200);")
            time.sleep(3)  # longer wait every 40 blocks
            
        try:
            # Re-fetch container & blocks each time
            container = wait.until(
                EC.presence_of_element_located((By.CLASS_NAME, "cr-product-container-icon"))
            )
            blocks = container.find_elements(By.CLASS_NAME, "cr-lottie-block")

            block = blocks[index]
            block.click()
            print(f"Clicked Lottie block {index + 1} ✅")

            # 🔑 Wait for modal to appear before looking for download button
            wait.until(
                EC.visibility_of_element_located((
                    By.XPATH,
                    "//button[@class='cr-btn cr-btn-default w-49' and normalize-space(text())='Download']"
                ))
            )
            time.sleep(0.5)

            # Download button
            download_btn = driver.find_element(
                By.XPATH,
                "//button[@class='cr-btn cr-btn-default w-49' and normalize-space(text())='Download']"
            )
            js_click(download_btn)
            print("Clicked the Download button ✅")
            time.sleep(1)

            # Download now button
            download_now_btn = wait.until(
                EC.visibility_of_element_located((
                    By.XPATH,
                    "//button[@class='cr-btn cr-btn-default w-100 mb-2' and .//span[normalize-space(text())='Download now']]"
                ))
            )
            js_click(download_now_btn)
            print("Clicked the 'Download now' button ✅")
            time.sleep(2)

            # Close first modal
            first_close = wait.until(
                EC.visibility_of_element_located((By.CLASS_NAME, "close"))
            )
            js_click(first_close)
            print("✅ Closed first modal (class='close')")
            time.sleep(1)

            # Close second modal
            second_close = wait.until(
                EC.visibility_of_element_located((By.CLASS_NAME, "modal-item-close"))
            )
            js_click(second_close)
            print("✅ Closed second modal (class='modal-item-close')")
            time.sleep(1)

        except StaleElementReferenceException:
            print(f"⚠️ Stale element at block {index+1}, retrying...")
            continue

    # --- 2) After all blocks, check for Next button ---
    try:
        next_btn = wait.until(
            EC.presence_of_element_located((
                By.XPATH,
                "//button[normalize-space(text())='Next']"
            ))
        )

        if next_btn.get_attribute("disabled") == "disabled":
            print("🚫 Next button is disabled. Stopping loop.")
            break
        else:
            # next_btn.click()
            js_click(next_btn)
            print("➡️ Clicked Next button, moving to next page...")
            time.sleep(5)  # wait for next page to load
    except TimeoutException:
        print("❌ Next button not found. Stopping loop.")
        break

print("\n🎉 Finished processing all pages!")



Total blocks found on this page: 50
⚠️ Stale element at block 1, retrying...
Clicked Lottie block 2 ✅
Clicked the Download button ✅
Clicked the 'Download now' button ✅
✅ Closed first modal (class='close')
✅ Closed second modal (class='modal-item-close')
Clicked Lottie block 3 ✅
Clicked the Download button ✅
Clicked the 'Download now' button ✅
✅ Closed first modal (class='close')
✅ Closed second modal (class='modal-item-close')
Clicked Lottie block 4 ✅
Clicked the Download button ✅
Clicked the 'Download now' button ✅
✅ Closed first modal (class='close')
✅ Closed second modal (class='modal-item-close')
Clicked Lottie block 5 ✅
Clicked the Download button ✅
Clicked the 'Download now' button ✅
✅ Closed first modal (class='close')
✅ Closed second modal (class='modal-item-close')
Clicked Lottie block 6 ✅
Clicked the Download button ✅
Clicked the 'Download now' button ✅
✅ Closed first modal (class='close')
✅ Closed second modal (class='modal-item-close')
Clicked Lottie block 7 ✅
Clicked the 

KeyboardInterrupt: 